In [2]:
!wget /content/gita_gpt_tutorial_data.json

/content/gita_gpt_tutorial_data.json: Scheme missing.


In [3]:
import json

file_path = '/content/gita_gpt_tutorial_data.json'

try:
    with open(file_path, 'r') as f:
        data = json.load(f)
    print("File loaded successfully!")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File loaded successfully!


In [5]:
# read it in to inspect it
with open('/content/gita_gpt_tutorial_data.json', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1711


In [7]:
# let's look at the first 500 characters
print(text[:500])

[
  {
    "prompt": "Chapter 2, Verse 47:\nSanskrit: कर्मण्येवाधिकारस्ते मा फलेषु कदाचन ।\nEnglish:",
    "completion": " You have a right to perform your prescribed duty, but you are not entitled to the fruits of action.\n"
  },
  {
    "prompt": "Chapter 4, Verse 7:\nSanskrit: यदा यदा हि धर्मस्य ग्लानिर्भवति भारत ।\nEnglish:",
    "completion": " Whenever there is a decline in righteousness and an increase in unrighteousness, O Arjuna, at that time I manifest myself on earth.\n"
  },
  {
    "


In [8]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 ",.012345678:ABCDEGHILMOSVWY[\]abcdefghijklmnoprstuvwxy{}ंअउएकगचजडणतथदधनपफभमयरलवशषसहऽािीुूेो्।—
97


In [9]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[40, 41, 41, 1, 51, 40, 37, 49, 37]
hii there


In [10]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([1711]) torch.int64
tensor([30,  0,  1,  1, 57,  0,  1,  1,  1,  1,  2, 48, 49, 47, 45, 48, 51,  2,
        14,  1,  2, 17, 40, 33, 48, 51, 37, 49,  1,  7,  3,  1, 27, 37, 49, 50,
        37,  1,  9, 12, 14, 31, 46, 26, 33, 46, 50, 43, 49, 41, 51, 14,  1, 63,
        79, 94, 77, 68, 94, 78, 92, 81, 87, 72, 88, 63, 87, 79, 84, 94, 69, 92,
         1, 77, 87,  1, 75, 80, 92, 83, 90,  1, 63, 71, 87, 65, 73,  1, 95, 31,
        46, 19, 46, 39, 44, 41, 50, 40, 14,  2,  3,  0,  1,  1,  1,  1,  2, 35,
        47, 45, 48, 44, 37, 51, 41, 47, 46,  2, 14,  1,  2,  1, 29, 47, 52,  1,
        40, 33, 53, 37,  1, 33,  1, 49, 41, 39, 40, 51,  1, 51, 47,  1, 48, 37,
        49, 38, 47, 49, 45,  1, 56, 47, 52, 49,  1, 48, 49, 37, 50, 35, 49, 41,
        34, 37, 36,  1, 36, 52, 51, 56,  3,  1, 34, 52, 51,  1, 56, 47, 52,  1,
        33, 49, 37,  1, 46, 47, 51,  1, 37, 46, 51, 41, 51, 44, 37, 36,  1, 51,
        47,  1, 51, 40, 37,  1, 38, 49, 52, 41, 51, 50,  1, 47, 38,  1, 33, 35,
        5

In [11]:
# Let's now split up the data into train and validation sets
n = int(0.8*len(data)) # first 80% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8
train_data[:block_size+1]

tensor([30,  0,  1,  1, 57,  0,  1,  1,  1])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([30]) the target: 0
when input is tensor([30,  0]) the target: 1
when input is tensor([30,  0,  1]) the target: 1
when input is tensor([30,  0,  1,  1]) the target: 57
when input is tensor([30,  0,  1,  1, 57]) the target: 0
when input is tensor([30,  0,  1,  1, 57,  0]) the target: 1
when input is tensor([30,  0,  1,  1, 57,  0,  1]) the target: 1
when input is tensor([30,  0,  1,  1, 57,  0,  1,  1]) the target: 1


In [14]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[46, 50, 43, 49, 41, 51, 14,  1],
        [46, 19, 46, 39, 44, 41, 50, 40],
        [48, 51, 37, 49,  1,  9,  3,  1],
        [35, 47, 45, 48, 44, 37, 51, 41]])
targets:
torch.Size([4, 8])
tensor([[50, 43, 49, 41, 51, 14,  1, 61],
        [19, 46, 39, 44, 41, 50, 40, 14],
        [51, 37, 49,  1,  9,  3,  1, 27],
        [47, 45, 48, 44, 37, 51, 41, 47]])
----
when input is [46] the target: 50
when input is [46, 50] the target: 43
when input is [46, 50, 43] the target: 49
when input is [46, 50, 43, 49] the target: 41
when input is [46, 50, 43, 49, 41] the target: 51
when input is [46, 50, 43, 49, 41, 51] the target: 14
when input is [46, 50, 43, 49, 41, 51, 14] the target: 1
when input is [46, 50, 43, 49, 41, 51, 14, 1] the target: 61
when input is [46] the target: 19
when input is [46, 19] the target: 46
when input is [46, 19, 46] the target: 39
when input is [46, 19, 46, 39] the target: 44
when input is [46, 19, 46, 39, 44] the target: 41
when input

In [15]:
print(xb) # our input to the transformer

tensor([[46, 50, 43, 49, 41, 51, 14,  1],
        [46, 19, 46, 39, 44, 41, 50, 40],
        [48, 51, 37, 49,  1,  9,  3,  1],
        [35, 47, 45, 48, 44, 37, 51, 41]])


In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 97])
tensor(5.1871, grad_fn=<NllLossBackward0>)

vyथो5।uWEh]सhि8[tOवरop}nकcg[tक1शिI[[{p—OS}च,]Cf}गSaयCुuकM।ी5i80्i0LभगषvतB4ु0ofप,।धd
B,ीW
v574LYoAkwc


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [18]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.845884323120117


In [19]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ंरwथ[57ु]फप1ष।—डOn{ग8IफV थुrOग]1D—्orयणasगं]2तd6WूH1षअभंf।a
C[0{धएाद}pतVअBए aapस1ध्ड[]ghpसYचेIeCीyल"t6,]bx—D,ए }Vध6d
तV{eववCyuहयशंfन्चgथमmफMऽi8Iyय5v8WVW।मe्le[षnVOCत]6णG{रउBव1YनhषHlastण:अ48uyeस4उ1YतOC.ajधडनचA5iLचह0महwकArr4—पagड,Yजulyuद "}cचरदsuवूtBYध[C{iपडGधe4अ
मड\नiV oूेdडO8rVरA
1oवकपEhदनYऽC[[Sapसरडफ{yीSA3sाुbk5डnVुड5W7ंcy ेBewnG5यि2]dHमirAvरधcheधग7yBss:षऽननIग,ेdHउYंfanchधh0oु
\नEhDSयज3ऽ.e{Lwेrr—1Cwo]bhB5ूषूiू"fअ0chO40kमkवमlड4एयजre्,पMभाषjbB2aो।sH]kवW}if[ ो6अ\ध0r6oफ]gउfफ\hभग4लीW।Aयो,वOाजm\ऽYउI[


In [20]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [21]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [22]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [27]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = torch.bmm(wei.unsqueeze(0).repeat(B, 1, 1), x) # (B, T, T) @ (B, T, C) ----> (B, T, C)
# torch.allclose(xbow, xbow2)
print(torch.max(torch.abs(xbow2 - xbow)))

tensor(3.2363e-08)


In [28]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = torch.bmm(wei.unsqueeze(0).repeat(B, 1, 1), x)
# torch.allclose(xbow, xbow3)
print(torch.max(torch.abs(xbow3 - xbow)))

tensor(3.2363e-08)


In [29]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [30]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [31]:
k.var()

tensor(1.0449)

In [32]:
wei.var()

tensor(1.0918)

In [33]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [34]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [35]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [36]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [37]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [38]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)


with open('/content/gita_gpt_tutorial_data.json', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.213857 M parameters
step 0: train loss 4.7992, val loss 4.8295
step 100: train loss 2.1599, val loss 2.6655
step 200: train loss 1.4854, val loss 2.6201
step 300: train loss 1.0195, val loss 2.7588
step 400: train loss 0.6588, val loss 3.1513
step 500: train loss 0.4195, val loss 3.5315
step 600: train loss 0.2955, val loss 3.9011
step 700: train loss 0.2455, val loss 3.9599
step 800: train loss 0.2096, val loss 4.2305
step 900: train loss 0.1950, val loss 4.5261
step 1000: train loss 0.1982, val loss 4.4139
step 1100: train loss 0.1797, val loss 4.5042
step 1200: train loss 0.1666, val loss 4.4325
step 1300: train loss 0.1634, val loss 4.5403
step 1400: train loss 0.1609, val loss 4.6276
step 1500: train loss 0.1660, val loss 4.6549
step 1600: train loss 0.1594, val loss 4.6488
step 1700: train loss 0.1558, val loss 4.5739
step 1800: train loss 0.1556, val loss 4.7067
step 1900: train loss 0.1516, val loss 4.8504
step 2000: train loss 0.1508, val loss 4.7005
step 2100: train loss 0.

In [41]:
import gradio as gr

def respond(prompt):
    system_prompt = "Answer in the spirit of the Bhagavad Gita. "
    final_prompt = system_prompt + prompt
    idx = torch.tensor([encode(final_prompt)], dtype=torch.long, device=device)
    output_idx = m.generate(idx, max_new_tokens=200)[0].tolist()
    output_text = decode(output_idx)
    # Remove the system prompt from the output
    response = output_text[len(final_prompt):].strip()
    return response

iface = gr.Interface(fn=respond,
                     inputs=gr.Textbox(lines=2, placeholder="Ask your question..."),
                     outputs="text",
                     title="Gita-GPT Mini",
                     description="Ask spiritual or philosophical questions and get responses inspired by the Bhagavad Gita (trained with a nanoGPT-style transformer).")

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b09b4a154ec4c63e2f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 